In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, cohen_kappa_score)
import warnings
warnings.filterwarnings('ignore')

In [6]:
# ==================== CONFIGURAÇÕES ====================
ARQUIVO_TRAIN = 'cvd/train.csv'
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [7]:
# ==================== FUNÇÕES AUXILIARES ====================

def calcular_metricas(y_true, y_pred):
    """Calcula todas as métricas de avaliação"""
    return {
        'Acurácia': accuracy_score(y_true, y_pred),
        'Precisão': precision_score(y_true, y_pred, zero_division=0),
        'Recall': recall_score(y_true, y_pred, zero_division=0),
        'F1-Score': f1_score(y_true, y_pred, zero_division=0),
        'Kappa': cohen_kappa_score(y_true, y_pred)
    }

def imprimir_resultados(nome_metodo, metricas_lista):
    """Imprime resultados formatados com média e desvio padrão"""
    print(f"\n{'='*60}")
    print(f"RESULTADOS: {nome_metodo}")
    print(f"{'='*60}")
    
    # Converter lista de dicionários em DataFrame para facilitar cálculos
    df_metricas = pd.DataFrame(metricas_lista)
    
    print(f"\n{'Métrica':<15} {'Média':<12} {'Desvio Padrão':<15}")
    print(f"{'-'*42}")
    
    for metrica in df_metricas.columns:
        media = df_metricas[metrica].mean()
        std = df_metricas[metrica].std()
        print(f"{metrica:<15} {media:.4f}       {std:.4f}")
    
    print(f"\nNúmero de execuções: {len(metricas_lista)}")

def preprocessar_dados(df):
    """Preprocessa o dataset: trata datas, valores ausentes e codifica variáveis"""
    df_proc = df.copy()
    
    # Remover colunas não necessárias
    colunas_remover = ['id', 'location', 'sym_on', 'hosp_vis']
    df_proc = df_proc.drop(columns=[col for col in colunas_remover if col in df_proc.columns])
    
    # Codificar variáveis categóricas
    le_country = LabelEncoder()
    le_gender = LabelEncoder()
    
    if 'country' in df_proc.columns:
        df_proc['country'] = le_country.fit_transform(df_proc['country'].astype(str))
    
    if 'gender' in df_proc.columns:
        df_proc['gender'] = le_gender.fit_transform(df_proc['gender'].astype(str))
    
    # Codificar colunas de sintomas (symptom1, symptom2, etc.)
    symptom_cols = [col for col in df_proc.columns if col.startswith('symptom')]
    for col in symptom_cols:
        if col in df_proc.columns:
            # Converter sintomas em variáveis categóricas numéricas
            le_symptom = LabelEncoder()
            # Substituir NA por string antes de codificar
            df_proc[col] = df_proc[col].fillna('no_symptom')
            df_proc[col] = le_symptom.fit_transform(df_proc[col].astype(str))
    
    # Tratar valores ausentes
    # Para colunas numéricas, preencher com a mediana
    numeric_cols = df_proc.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        df_proc[col] = df_proc[col].fillna(df_proc[col].median())
    
    # Para colunas categóricas, preencher com a moda e depois codificar
    categorical_cols = df_proc.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        df_proc[col] = df_proc[col].fillna(df_proc[col].mode()[0] if len(df_proc[col].mode()) > 0 else 'unknown')
        # Codificar se ainda houver colunas categóricas
        if df_proc[col].dtype == 'object':
            le = LabelEncoder()
            df_proc[col] = le.fit_transform(df_proc[col].astype(str))
    
    return df_proc


In [8]:
# ==================== CARREGAR E PREPARAR DADOS ====================

print("="*60)
print("CLASSIFICAÇÃO KNN - COVID-19 DATASET")
print("="*60)

print("\nCarregando dataset de treino...")
df = pd.read_csv(ARQUIVO_TRAIN)

print(f"Dataset carregado: {df.shape[0]} amostras, {df.shape[1]} features")

# Criar coluna 'result' baseada em 'death' e 'recov'
# result = 1 se morte (death=1), result = 0 se recuperado ou sem informação
if 'death' in df.columns:
    df['result'] = df['death'].apply(lambda x: 1 if x == 1 else 0)
    print(f"\nDistribuição da classe alvo (result - morreu ou não):")
    print(f"  Recuperados (0): {(df['result'] == 0).sum()}")
    print(f"  Falecidos (1): {(df['result'] == 1).sum()}")
else:
    print("\nERRO: Coluna 'death' não encontrada no dataset!")
    exit()

# Remover linhas onde não há informação de death
if 'death' in df.columns:
    df = df[df['death'].notna()].copy()
if 'recov' in df.columns:
    df = df[df['recov'].notna() | df['death'].notna()].copy()

# Remover colunas 'death' e 'recov' das features
colunas_remover_target = []
if 'death' in df.columns:
    colunas_remover_target.append('death')
if 'recov' in df.columns:
    colunas_remover_target.append('recov')

if colunas_remover_target:
    df = df.drop(colunas_remover_target, axis=1)

# Preprocessar dados
df_processado = preprocessar_dados(df)

# Separar features e target
X = df_processado.drop('result', axis=1)
y = df_processado['result']

print(f"\nDataset após preprocessamento:")
print(f"  Amostras: {X.shape[0]}")
print(f"  Features: {X.shape[1]}")
print(f"  Features utilizadas: {list(X.columns)}")

# Verificar se há dados suficientes
if len(X) < 10:
    print("\nERRO: Dataset muito pequeno para análise!")
    exit()

# ==================== DIVISÃO TRAIN/TEST (70/30) ====================

print(f"\n{'='*60}")
print("DIVISÃO DOS DADOS: TRAIN/TEST (70/30)")
print(f"{'='*60}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=RANDOM_STATE, stratify=y
)

print(f"\nTamanho dos conjuntos:")
print(f"  - Treino: {len(X_train)} amostras ({len(X_train)/len(X)*100:.1f}%)")
print(f"  - Teste: {len(X_test)} amostras ({len(X_test)/len(X)*100:.1f}%)")

# Normalizar dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ==================== MÉTODO 1: HOLD-OUT (70/30) ====================

print(f"\n\n{'='*60}")
print("MÉTODO 1: HOLD-OUT (70/30)")
print(f"{'='*60}")

# Repetir hold-out 30 vezes para calcular desvio padrão
metricas_holdout = []

for i in range(30):
    # Nova divisão a cada iteração
    X_train_ho, X_test_ho, y_train_ho, y_test_ho = train_test_split(
        X, y, test_size=0.3, random_state=i, stratify=y
    )
    
    scaler_ho = StandardScaler()
    X_train_ho_scaled = scaler_ho.fit_transform(X_train_ho)
    X_test_ho_scaled = scaler_ho.transform(X_test_ho)
    
    # Treinar KNN com valor padrão (k=5)
    knn = KNeighborsClassifier()
    knn.fit(X_train_ho_scaled, y_train_ho)
    y_pred = knn.predict(X_test_ho_scaled)
    
    # Calcular métricas
    metricas = calcular_metricas(y_test_ho, y_pred)
    metricas_holdout.append(metricas)

imprimir_resultados("Hold-Out (70/30)", metricas_holdout)

# ==================== MÉTODO 2: K-FOLD (K=5) ====================

print(f"\n\n{'='*60}")
print("MÉTODO 2: K-FOLD CROSS-VALIDATION (K=5)")
print(f"{'='*60}")

kfold = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
metricas_kfold = []

# Normalizar todo o dataset
scaler_full = StandardScaler()
X_scaled_full = scaler_full.fit_transform(X)

fold_num = 1
for train_idx, test_idx in kfold.split(X_scaled_full, y):
    X_train_kf = X_scaled_full[train_idx]
    X_test_kf = X_scaled_full[test_idx]
    y_train_kf = y.iloc[train_idx]
    y_test_kf = y.iloc[test_idx]
    
    # Treinar KNN com valor padrão (k=5)
    knn = KNeighborsClassifier()
    knn.fit(X_train_kf, y_train_kf)
    y_pred = knn.predict(X_test_kf)
    
    # Calcular métricas
    metricas = calcular_metricas(y_test_kf, y_pred)
    metricas_kfold.append(metricas)
    
    print(f"Fold {fold_num}/5 concluído")
    fold_num += 1

imprimir_resultados("K-Fold (K=5)", metricas_kfold)

# ==================== MÉTODO 3: LEAVE-ONE-OUT ====================

print(f"\n\n{'='*60}")
print("MÉTODO 3: LEAVE-ONE-OUT CROSS-VALIDATION")
print(f"{'='*60}")

loo = LeaveOneOut()
metricas_loo = []

# Para LOO, vamos fazer por batches para calcular métricas
y_true_total = []
y_pred_total = []

print(f"Processando {len(X)} iterações...")

iteracao = 0
for train_idx, test_idx in loo.split(X_scaled_full):
    X_train_loo = X_scaled_full[train_idx]
    X_test_loo = X_scaled_full[test_idx]
    y_train_loo = y.iloc[train_idx]
    y_test_loo = y.iloc[test_idx]
    
    # Treinar KNN com valor padrão (k=5)
    knn = KNeighborsClassifier()
    knn.fit(X_train_loo, y_train_loo)
    y_pred = knn.predict(X_test_loo)
    
    y_true_total.extend(y_test_loo)
    y_pred_total.extend(y_pred)
    
    iteracao += 1
    if iteracao % 50 == 0:
        print(f"  Progresso: {iteracao}/{len(X)}")

# Calcular métricas globais do LOO
metricas_loo_global = calcular_metricas(y_true_total, y_pred_total)
metricas_loo.append(metricas_loo_global)

# Para LOO, o desvio padrão é 0 pois é uma única execução completa
print(f"\n{'='*60}")
print(f"RESULTADOS: Leave-One-Out")
print(f"{'='*60}")

print(f"\n{'Métrica':<15} {'Valor':<12} {'Desvio Padrão':<15}")
print(f"{'-'*42}")

for metrica, valor in metricas_loo_global.items():
    print(f"{metrica:<15} {valor:.4f}       0.0000")

print(f"\nNúmero de execuções: {len(X)} (uma por amostra)")

# ==================== RESUMO FINAL ====================

print(f"\n\n{'#'*60}")
print("RESUMO COMPARATIVO DOS MÉTODOS")
print(f"{'#'*60}")

print(f"\nValor de K utilizado: 5 (padrão)")
print(f"Dataset: {len(X)} amostras, {X.shape[1]} features")
print(f"\nMétodo           Acurácia(μ±σ)    F1-Score(μ±σ)    Precisão(μ±σ)")
print(f"{'-'*70}")

# Hold-out
df_ho = pd.DataFrame(metricas_holdout)
print(f"Hold-Out         {df_ho['Acurácia'].mean():.4f}±{df_ho['Acurácia'].std():.4f}   "
      f"{df_ho['F1-Score'].mean():.4f}±{df_ho['F1-Score'].std():.4f}   "
      f"{df_ho['Precisão'].mean():.4f}±{df_ho['Precisão'].std():.4f}")

# K-Fold
df_kf = pd.DataFrame(metricas_kfold)
print(f"K-Fold (5)       {df_kf['Acurácia'].mean():.4f}±{df_kf['Acurácia'].std():.4f}   "
      f"{df_kf['F1-Score'].mean():.4f}±{df_kf['F1-Score'].std():.4f}   "
      f"{df_kf['Precisão'].mean():.4f}±{df_kf['Precisão'].std():.4f}")

# LOO
print(f"Leave-One-Out    {metricas_loo_global['Acurácia']:.4f}±0.0000   "
      f"{metricas_loo_global['F1-Score']:.4f}±0.0000   "
      f"{metricas_loo_global['Precisão']:.4f}±0.0000")

print(f"\n{'Métrica':<15} {'Hold-Out':<12} {'K-Fold':<12} {'LOO':<12}")
print(f"{'-'*55}")
print(f"{'Recall':<15} {df_ho['Recall'].mean():.4f}±{df_ho['Recall'].std():.4f}   "
      f"{df_kf['Recall'].mean():.4f}±{df_kf['Recall'].std():.4f}   "
      f"{metricas_loo_global['Recall']:.4f}±0.0000")
print(f"{'Kappa':<15} {df_ho['Kappa'].mean():.4f}±{df_ho['Kappa'].std():.4f}   "
      f"{df_kf['Kappa'].mean():.4f}±{df_kf['Kappa'].std():.4f}   "
      f"{metricas_loo_global['Kappa']:.4f}±0.0000")


# ==================== MÉTODO 2: K-FOLD (K=5) ====================

print(f"\n\n{'='*60}")
print("MÉTODO 2: K-FOLD CROSS-VALIDATION (K=5)")
print(f"{'='*60}")

kfold = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
metricas_kfold = []

# Normalizar todo o dataset
scaler_full = StandardScaler()
X_scaled_full = scaler_full.fit_transform(X)

fold_num = 1
for train_idx, test_idx in kfold.split(X_scaled_full, y):
    X_train_kf = X_scaled_full[train_idx]
    X_test_kf = X_scaled_full[test_idx]
    y_train_kf = y.iloc[train_idx]
    y_test_kf = y.iloc[test_idx]
    
    # Treinar KNN com valor padrão (k=5)
    knn = KNeighborsClassifier()
    knn.fit(X_train_kf, y_train_kf)
    y_pred = knn.predict(X_test_kf)
    
    # Calcular métricas
    metricas = calcular_metricas(y_test_kf, y_pred)
    metricas_kfold.append(metricas)
    
    print(f"Fold {fold_num}/5 concluído")
    fold_num += 1

imprimir_resultados("K-Fold (K=5)", metricas_kfold)

# ==================== MÉTODO 3: STRATIFIED K-FOLD (K=5) ====================

print(f"\n\n{'='*60}")
print("MÉTODO 3: STRATIFIED K-FOLD CROSS-VALIDATION (K=5)")
print(f"{'='*60}")

from sklearn.model_selection import StratifiedKFold

skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
metricas_skfold = []

print("Estratificação - proporção de classes mantida em cada fold:")
print(f"  Dataset completo: Classe 0 = {(y==0).sum()} ({(y==0).sum()/len(y)*100:.1f}%), "
      f"Classe 1 = {(y==1).sum()} ({(y==1).sum()/len(y)*100:.1f}%)\n")

fold_num = 1
for train_idx, test_idx in skfold.split(X_scaled_full, y):
    X_train_skf = X_scaled_full[train_idx]
    X_test_skf = X_scaled_full[test_idx]
    y_train_skf = y.iloc[train_idx]
    y_test_skf = y.iloc[test_idx]
    
    # Mostrar distribuição das classes em cada fold
    print(f"Fold {fold_num}/5: Treino -> Classe 0 = {(y_train_skf==0).sum()} ({(y_train_skf==0).sum()/len(y_train_skf)*100:.1f}%), "
          f"Classe 1 = {(y_train_skf==1).sum()} ({(y_train_skf==1).sum()/len(y_train_skf)*100:.1f}%)")
    
    # Treinar KNN com valor padrão (k=5)
    knn = KNeighborsClassifier()
    knn.fit(X_train_skf, y_train_skf)
    y_pred = knn.predict(X_test_skf)
    
    # Calcular métricas
    metricas = calcular_metricas(y_test_skf, y_pred)
    metricas_skfold.append(metricas)
    
    fold_num += 1

imprimir_resultados("Stratified K-Fold (K=5)", metricas_skfold)

print(f"\n{'#'*60}")
print("ANÁLISE CONCLUÍDA")
print(f"{'#'*60}")

CLASSIFICAÇÃO KNN - COVID-19 DATASET

Carregando dataset de treino...
Dataset carregado: 222 amostras, 16 features

Distribuição da classe alvo (result - morreu ou não):
  Recuperados (0): 159
  Falecidos (1): 63

Dataset após preprocessamento:
  Amostras: 222
  Features: 11
  Features utilizadas: ['country', 'gender', 'age', 'vis_wuhan', 'from_wuhan', 'symptom1', 'symptom2', 'symptom3', 'symptom4', 'symptom5', 'symptom6']

DIVISÃO DOS DADOS: TRAIN/TEST (70/30)

Tamanho dos conjuntos:
  - Treino: 155 amostras (69.8%)
  - Teste: 67 amostras (30.2%)


MÉTODO 1: HOLD-OUT (70/30)

RESULTADOS: Hold-Out (70/30)

Métrica         Média        Desvio Padrão  
------------------------------------------
Acurácia        0.8806       0.0328
Precisão        0.8396       0.0666
Recall          0.7193       0.1000
F1-Score        0.7711       0.0692
Kappa           0.6914       0.0894

Número de execuções: 30


MÉTODO 2: K-FOLD CROSS-VALIDATION (K=5)
Fold 1/5 concluído
Fold 2/5 concluído
Fold 3/5 conc